# 1 - Web Scraping / Scrapeo con BeatifulSoup (Bs4)
<p align = 'center'>
<img src = 'https://editor.analyticsvidhya.com/uploads/75676cover.jfif'/>
</p>

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web de manera automatizada y masiva. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs.

En los últimos años el web scraping se ha convertido en una **técnica muy utilizada** dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro recurso a **falta de una API o un feed RSS**. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

Todo aquello que necesite interactividad con la página web va a necesitar **selenium**, una librería que nos permite darle instrucciones a nuestro explorador para poder simular los movimientos que haríamos por dentro de la web:
 - Cambiar de página
 - Rellenar formularios
 - Hacer click en botones
 - Cerrar pop-ups
 - Rechazar cookies
 
Por otro lado, todo aquello que sea estático podemos tirar únicamente de **BeautifulSoup**, la primera librería para scrapeo que vamos a conocer que nos permite, una vez cargada la página de la que vamos a extraer la información, recorrer el código HTML y quedarnos con aquellos datos que necesitemos.

Tendremos que realizar scrapeo cuando no dispongamos de una fuente más sencilla de la que obtener información como una API, una base de datos, fichero, etc...

# 2 - Extracción de los datos del HTML

Para scrapear necesitamos saber que pinta tiene la **estructura general** que tiene un HTML.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`, `<span>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$


Las etiquetas pueden tener atributos. Por ejemplo:

`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>`

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página, varios elementos pueden tener la misma clase.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca, no puede haber dos etiquetas con el mismo id.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las **etiquetas** que nos interesan dentro de todo el HTML de la página web.

Para ello vamos a usar **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests as req
from bs4 import BeautifulSoup as bs #este alias es standard

# 3 - Scrapeando **WIKIPEDIA**

Vamos a echar un ojo a esta web **[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)** para intentar sacar la información de la tabla que contiene la esperanza de vida de los países europeos.

Hasta ahora habíamos visto `requests`para hacer llamadas a url (normalmente endpoints de las APIS que hemos estado mirando)

¿Y si llamamos a una url de una web digamos `normal`?

In [65]:
#url de la pagina a scrapear
url = 'https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

html = req.get(url).text #obtenemos el contenido de la pagina
html[:1000] #mostramos los primeros 1000 caracteres

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of European countries by life expectancy - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-m

In [66]:
str(html).split('/head>')[1][:1000] #mostramos los primeros 1000 caracteres de la seccion body / Esto es una gualtrapada para ver el codigo html de la pagina

'\n<body class="skin--responsive skin-vector skin-vector-search-vue mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject mw-editable page-List_of_European_countries_by_life_expectancy rootpage-List_of_European_countries_by_life_expectancy skin-vector-2022 action-view"><a class="mw-jump-link" href="#bodyContent">Jump to content</a>\n<div class="vector-header-container">\n\t<header class="vector-header mw-header">\n\t\t<div class="vector-header-start">\n\t\t\t<nav class="vector-main-menu-landmark" aria-label="Site">\n\t\t\t\t\n<div id="vector-main-menu-dropdown" class="vector-dropdown vector-main-menu-dropdown vector-button-flush-left vector-button-flush-right"  >\n\t<input type="checkbox" id="vector-main-menu-dropdown-checkbox" role="button" aria-haspopup="true" data-event-name="ui.dropdown-vector-main-menu-dropdown" class="vector-dropdown-checkbox "  aria-label="Main menu"  >\n\t<label id="vector-main-menu-dropdown-label" for="vector-main-menu-dropdown-checkbox" class="vector-dr

Esto es infumable y para nada práctico. Para ello BeautifulSoup entra en juego para facilitarnos la vida.

Vimos en la docu, la manera de pasarle un `html` para que nos lo parsee (traduzca/convierta) en un objeto de BeautifulSoup muchos más manejable, amigable y con muchos métodos que nos facilitan el poder navegar el `html` para encontrar el contenido exacto que queremos extraer.

Así, al **parsearlo**, es mucho más **legible**. A este objeto por convención se le llama `soup`.

In [67]:
#Alla
soup = bs(html, 'html.parser') #creamos el objeto soup, y le decimos que lo que le pasamos tiene pinta de html
type(soup)

bs4.BeautifulSoup

Vamos a acceder a la tabla que hay en la web. 

En primer lugar es necesario entender cómo se estructura una tabla en código HTML.

Gracias a BeautifulSoup podemos encontrar los diferentes **elementos** de una manera muy **veloz**.

Hay varias funciones para encontrar elementos concretos. `find()` y `find_all()` son dos ejemplos.

Vamos a ver como es la **estructura general** de una tabla en html:

![](https://www.corelangs.com/html/tables/img/html-table-structure.png)

![](https://www.ourtutorials.in/html/img/table1.JPG)

In [68]:
# Tabla
tabla = soup.find('table') #buscamos la tabla, en este caso solo hay una. Si hubiera mas, solo nos devolveria la primera. Si queremos todas, usamos find_all
# soup.find_all('table')  -->  En caso de que tuviéramos más de una tabla esta sería la línea de código correcta para acceder a todas ellas
tabla

<table class="wikitable sortable mw-datatable static-row-numbers sticky-header-multi sort-under col1left col21right" style="text-align:center;">
<tbody><tr>
<th rowspan="2" style="vertical-align:middle;">Countries and<br/>territories
</th>
<th colspan="7">Life expectancy for population in general
</th>
<th colspan="4" style="border-left-width:2px;">Life expectancy for male
</th>
<th colspan="4" style="border-left-width:2px;">Life expectancy for female
</th>
<th colspan="4" style="border-left-width:2px;"><style data-mw-deduplicate="TemplateStyles:r1038841319">.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}</style><span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">Sex gap</span>
</th>
<th rowspan="2" style="border-left-width:2px; text-align:middle;"><link href="mw-data:TemplateStyles:r1038841319" rel="mw-deduplicated-inline-style"/><span class="rt-commentedText tooltip tooltip-dotted" title="Total popula

In [69]:
tabla.prettify()

'<table class="wikitable sortable mw-datatable static-row-numbers sticky-header-multi sort-under col1left col21right" style="text-align:center;">\n <tbody>\n  <tr>\n   <th rowspan="2" style="vertical-align:middle;">\n    Countries and\n    <br/>\n    territories\n   </th>\n   <th colspan="7">\n    Life expectancy for population in general\n   </th>\n   <th colspan="4" style="border-left-width:2px;">\n    Life expectancy for male\n   </th>\n   <th colspan="4" style="border-left-width:2px;">\n    Life expectancy for female\n   </th>\n   <th colspan="4" style="border-left-width:2px;">\n    <style data-mw-deduplicate="TemplateStyles:r1038841319">\n     .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}\n    </style>\n    <span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">\n     Sex gap\n    </span>\n   </th>\n   <th rowspan="2" style="border-left-width:2px; text-align:middle;">\n    <link href="mw-data:Templa

In [10]:
print(tabla.prettify()[:1000])

<table class="wikitable sortable mw-datatable static-row-numbers sticky-header-multi sort-under col1left col21right" style="text-align:center;">
 <tbody>
  <tr>
   <th rowspan="2" style="vertical-align:middle;">
    Countries and
    <br/>
    territories
   </th>
   <th colspan="7">
    Life expectancy for population in general
   </th>
   <th colspan="4" style="border-left-width:2px;">
    Life expectancy for male
   </th>
   <th colspan="4" style="border-left-width:2px;">
    Life expectancy for female
   </th>
   <th colspan="4" style="border-left-width:2px;">
    <style data-mw-deduplicate="TemplateStyles:r1038841319">
     .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}
    </style>
    <span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">
     Sex gap
    </span>
   </th>
   <th rowspan="2" style="border-left-width:2px; text-align:middle;">
    <link href="mw-data:TemplateStyles:r1038841319" rel="

In [70]:
# Filas de la tabla
filas = tabla.find_all('tr') # tr --> table row
filas #Acordaros que vimos que find all nos devolvía una lista con todos los elementos encontrados

[<tr>
 <th rowspan="2" style="vertical-align:middle;">Countries and<br/>territories
 </th>
 <th colspan="7">Life expectancy for population in general
 </th>
 <th colspan="4" style="border-left-width:2px;">Life expectancy for male
 </th>
 <th colspan="4" style="border-left-width:2px;">Life expectancy for female
 </th>
 <th colspan="4" style="border-left-width:2px;"><style data-mw-deduplicate="TemplateStyles:r1038841319">.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}</style><span class="rt-commentedText tooltip tooltip-dotted" title="Difference in life expectancy for female and male">Sex gap</span>
 </th>
 <th rowspan="2" style="border-left-width:2px; text-align:middle;"><link href="mw-data:TemplateStyles:r1038841319" rel="mw-deduplicated-inline-style"/><span class="rt-commentedText tooltip tooltip-dotted" title="Total population, as of 1 July 2023, in thousands">Population<br/>(thous.)</span>
 </th>
 <th rowspan="2">
 </th></tr>,
 <tr>
 <th style="vertical-align

In [12]:
type(filas) # ResultSet es una lista

bs4.element.ResultSet

Al ser una lista puedo recorrer cada elemento.

In [71]:
# Texto del primer elemento de la lista
filas[0].text

'\nCountries andterritories\n\nLife expectancy for population in general\n\nLife expectancy for male\n\nLife expectancy for female\n\nSex gap\n\nPopulation(thous.)\n\n\n'

In [63]:
print(filas[0].text)




Country

GNI percapita (US$)[2]

Year



In [15]:
# texto de cada fila solo para los 3 primeros elementos
for f in filas[:3]:
    print(type(f.text), f.text)
    print('-----------------')

<class 'str'> 
Countries andterritories

Life expectancy for population in general

Life expectancy for male

Life expectancy for female

Sex gap

Population(thous.)



-----------------
<class 'str'> 
at birth

bonus0→15

at 15

bonus15→65

at 65

bonus65→80

at 80

at birth

at 15

at 65

at 80

at birth

at 15

at 65

at 80

at birth

at 15

at 65

at 80

-----------------
<class 'str'> 
 Monaco
86.37
0.45
71.82
1.40
23.21
3.09
11.30
84.45
69.90
21.56
10.14
88.50
73.95
25.05
12.50
4.06
4.05
3.49
2.36
39


-----------------


In [16]:
#vamos a sacar el texto de 3 primeros elementos del cuerpo de la tabla
for f in filas[2:5]:
    print(type(f.text), f.text)
    print('-----------------')

<class 'str'> 
 Monaco
86.37
0.45
71.82
1.40
23.21
3.09
11.30
84.45
69.90
21.56
10.14
88.50
73.95
25.05
12.50
4.06
4.05
3.49
2.36
39


-----------------
<class 'str'> 
 San Marino
85.71
0.15
70.86
1.78
22.64
2.88
10.52
84.21
69.36
21.25
9.52
87.10
72.24
23.90
11.31
2.89
2.88
2.65
1.79
34


-----------------
<class 'str'> 
 Andorra
84.04
0.58
69.62
1.85
21.47
3.72
10.19
82.10
67.68
19.81
9.03
86.11
71.70
23.18
11.16
4.01
4.02
3.36
2.13
81


-----------------


In [17]:
f.text.split('\n')

['',
 '\xa0Andorra',
 '84.04',
 '0.58',
 '69.62',
 '1.85',
 '21.47',
 '3.72',
 '10.19',
 '82.10',
 '67.68',
 '19.81',
 '9.03',
 '86.11',
 '71.70',
 '23.18',
 '11.16',
 '4.01',
 '4.02',
 '3.36',
 '2.13',
 '81',
 '',
 '']

In [18]:
# Hagamos una lista, que contenga una lista por fila
# strip() --> quita los espacios que haya delante y detrás del texto
# split() --> va a separar la lista por la secuencia de caracteres '\n' para hacer una lista
filas_limpias = [f.text.strip().split('\n') for f in filas]
filas_limpias[:5]

[['Countries andterritories',
  '',
  'Life expectancy for population in general',
  '',
  'Life expectancy for male',
  '',
  'Life expectancy for female',
  '',
  'Sex gap',
  '',
  'Population(thous.)'],
 ['at birth',
  '',
  'bonus0→15',
  '',
  'at 15',
  '',
  'bonus15→65',
  '',
  'at 65',
  '',
  'bonus65→80',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80',
  '',
  'at birth',
  '',
  'at 15',
  '',
  'at 65',
  '',
  'at 80'],
 ['Monaco',
  '86.37',
  '0.45',
  '71.82',
  '1.40',
  '23.21',
  '3.09',
  '11.30',
  '84.45',
  '69.90',
  '21.56',
  '10.14',
  '88.50',
  '73.95',
  '25.05',
  '12.50',
  '4.06',
  '4.05',
  '3.49',
  '2.36',
  '39'],
 ['San Marino',
  '85.71',
  '0.15',
  '70.86',
  '1.78',
  '22.64',
  '2.88',
  '10.52',
  '84.21',
  '69.36',
  '21.25',
  '9.52',
  '87.10',
  '72.24',
  '23.90',
  '11.31',
  '2.89',
  '2.88',
  '2.65',
  '1.79',
  '34'],


In [19]:
# La primera fila es la cabecera, y hay que limpiar los espacios en blanco
cabecera = filas_limpias[0]
cabecera

['Countries andterritories',
 '',
 'Life expectancy for population in general',
 '',
 'Life expectancy for male',
 '',
 'Life expectancy for female',
 '',
 'Sex gap',
 '',
 'Population(thous.)']

In [20]:
# vamos a arreglarlo
final = [] # aquí el resultado final limpio
for fila in filas_limpias:
    tmp = [] # lista vacía temporal
    for elemento in fila:
        if elemento != '':
            tmp.append(elemento)
    final.append(tmp)

final[:5]

[['Countries andterritories',
  'Life expectancy for population in general',
  'Life expectancy for male',
  'Life expectancy for female',
  'Sex gap',
  'Population(thous.)'],
 ['at birth',
  'bonus0→15',
  'at 15',
  'bonus15→65',
  'at 65',
  'bonus65→80',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80',
  'at birth',
  'at 15',
  'at 65',
  'at 80'],
 ['Monaco',
  '86.37',
  '0.45',
  '71.82',
  '1.40',
  '23.21',
  '3.09',
  '11.30',
  '84.45',
  '69.90',
  '21.56',
  '10.14',
  '88.50',
  '73.95',
  '25.05',
  '12.50',
  '4.06',
  '4.05',
  '3.49',
  '2.36',
  '39'],
 ['San Marino',
  '85.71',
  '0.15',
  '70.86',
  '1.78',
  '22.64',
  '2.88',
  '10.52',
  '84.21',
  '69.36',
  '21.25',
  '9.52',
  '87.10',
  '72.24',
  '23.90',
  '11.31',
  '2.89',
  '2.88',
  '2.65',
  '1.79',
  '34'],
 ['Andorra',
  '84.04',
  '0.58',
  '69.62',
  '1.85',
  '21.47',
  '3.72',
  '10.19',
  '82.10',
  '67.68',
  '19.81',
  '9.03',
  '86.11

Hasta ahora hemos cogido la información contenida en una página que tiene una tabla. Hemos identificado la tabla y hemos accedido a ella. Con `find()` hemos buscado la etiqueta que nos interesa para ello (`<table>`). Una vez obtenido el HTML únicamente de la tabla lo hemos traducido para que sea algo más legible y hemos arreglado su contenido para poder llegar al siguiente paso: crear nuestro DataFrame.

In [21]:
final[0] # primer elemento de la lista final

['Countries andterritories',
 'Life expectancy for population in general',
 'Life expectancy for male',
 'Life expectancy for female',
 'Sex gap',
 'Population(thous.)']

In [22]:
final[1] # segundo elemento de la lista final

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [23]:
final[2]

['Monaco',
 '86.37',
 '0.45',
 '71.82',
 '1.40',
 '23.21',
 '3.09',
 '11.30',
 '84.45',
 '69.90',
 '21.56',
 '10.14',
 '88.50',
 '73.95',
 '25.05',
 '12.50',
 '4.06',
 '4.05',
 '3.49',
 '2.36',
 '39']

Para poder crear el DataFrame a partir de listas es necesario que todas ellas tengan la **misma cantidad de elementos**, tanto la que dará nombre a las columnas como la que contendrá la información de las filas.

In [4]:
import pandas as pd

In [24]:
nombres_columnas = final[1] # la primera fila la omitimos, nos quedamos la segunda, pero tenemos que arreglarla
nombres_columnas

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [25]:
data = final[2:] # desde la primera fila con información, la de Liechtenstein, hasta el final

In [ ]:
len(data[0]), len(nombres_columnas) # las filas tienen info para más columnas que la lista que dará nombre a las columnas...

En este caso no coinciden. Así que tendremos que hacer algún pequeño arreglo extra antes de conseguir hacer el DataFrame.

In [26]:
nombres_columnas

['at birth',
 'bonus0→15',
 'at 15',
 'bonus15→65',
 'at 65',
 'bonus65→80',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80',
 'at birth',
 'at 15',
 'at 65',
 'at 80']

In [27]:
data[0]

['Monaco',
 '86.37',
 '0.45',
 '71.82',
 '1.40',
 '23.21',
 '3.09',
 '11.30',
 '84.45',
 '69.90',
 '21.56',
 '10.14',
 '88.50',
 '73.95',
 '25.05',
 '12.50',
 '4.06',
 '4.05',
 '3.49',
 '2.36',
 '39']

In [28]:
nombres_columnas.insert(0, 'Country') # al añadir en primer lugar 'Country' ya tenemos tantos elementos como en el resto, que serán las filas

In [29]:
df = pd.DataFrame(data, columns = nombres_columnas)

ValueError: 20 columns passed, passed data had 21 columns

In [30]:
df.head(10)

NameError: name 'df' is not defined

In [ ]:
#RETO

#Vamos a escrapear la tabla de esta web --> https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita
#Escrapeamos la primera tabla, y cuando lo tengamos vamos con la segunda
#La salida debe ser un dataframe limpito,

In [2]:
#Aqui
import requests as req
from bs4 import BeautifulSoup as bs 

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GNI_(nominal)_per_capita'
html = req.get(url).text
html

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of countries by GNI (nominal) per capita - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature

In [19]:
sopa = bs(html, 'html.parser')
sopa

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of countries by GNI (nominal) per capita - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-lim

In [20]:
tablas = sopa.find_all('table')
tablas

[<table class="box-More_citations_needed plainlinks metadata ambox ambox-content ambox-Refimprove" role="presentation"><tbody><tr><td class="mbox-image"><div class="mbox-image-div"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/File:Question_book-new.svg"><img alt="" class="mw-file-element" data-file-height="399" data-file-width="512" decoding="async" height="39" src="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/50px-Question_book-new.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/75px-Question_book-new.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/100px-Question_book-new.svg.png 2x" width="50"/></a></span></div></td><td class="mbox-text"><div class="mbox-text-span">This article <b>needs additional citations for <a href="/wiki/Wikipedia:Verifiability" title="Wikipedia:Verifiability">verification</a></b>.<span class="hide-when-compact"> Please help <a href="/wiki/Specia

In [21]:
tabla_buena = tablas[1]
print(tabla_buena.prettify())

<table class="wikitable sortable sticky-header sort-under col2left" style="text-align:right">
 <tbody>
  <tr>
   <th>
   </th>
   <th>
    Country
   </th>
   <th>
    GNI per
    <br/>
    capita (US$)
    <sup class="reference" id="cite_ref-2">
     <a href="#cite_note-2">
      <span class="cite-bracket">
       [
      </span>
      2
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th>
    Year
   </th>
  </tr>
  <tr>
   <td>
   </td>
   <td>
    <b>
     High Income Economies
    </b>
   </td>
   <td>
   </td>
   <td>
    2023
   </td>
  </tr>
  <tr>
   <td>
    –
   </td>
   <td>
    <i>
     <span class="flagicon">
      <span class="mw-image-border" typeof="mw:File">
       <span>
        <img alt="" class="mw-file-element" data-file-height="500" data-file-width="1000" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/23px-Flag_of_Bermuda.svg.png" srcset="//upload.wikimedia.org/wi

In [23]:
filas = tabla_buena.find_all('tr')
filas

[<tr>
 <th>
 </th>
 <th>Country
 </th>
 <th>GNI per<br/>capita (US$)<sup class="reference" id="cite_ref-2"><a href="#cite_note-2"><span class="cite-bracket">[</span>2<span class="cite-bracket">]</span></a></sup>
 </th>
 <th>Year
 </th></tr>,
 <tr>
 <td>
 </td>
 <td><b>High Income Economies</b>
 </td>
 <td>
 </td>
 <td>2023
 </td></tr>,
 <tr>
 <td>–</td>
 <td><i><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="500" data-file-width="1000" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/23px-Flag_of_Bermuda.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/35px-Flag_of_Bermuda.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Flag_of_Bermuda.svg/46px-Flag_of_Bermuda.svg.png 2x" width="23"/></span></span> </span><a href="/wiki/Bermuda" title="Bermuda">Bermuda</a></i> (UK)</td>
 <td>134,640</td>
 <td

In [39]:
filas_buenas = []
for f in filas:
    filas_buenas.append(f.text.strip().split('\n'))
    


In [40]:
filas_buenas

[['Country', '', 'GNI percapita (US$)[2]', '', 'Year'],
 ['High Income Economies', '', '', '', '2023'],
 ['–', '\xa0Bermuda (UK)', '134,640', '2022'],
 ['1', '\xa0Liechtenstein', '116,600', '2009'],
 ['2', '\xa0Norway', '102,460', '2023'],
 ['3', '\xa0\xa0Switzerland', '95,160', '2023'],
 ['4', '\xa0Luxembourg', '88,370', '2023'],
 ['5', '\xa0Ireland', '80,390', '2023'],
 ['6', '\xa0United States', '80,300', '2023'],
 ['7', '\xa0Iceland', '79,840', '2023'],
 ['–', '\xa0Isle of Man (UK)', '76,300', '2021'],
 ['–', '\xa0Faroe Islands (Denmark)', '74,420', '2022'],
 ['8', '\xa0Denmark', '73,360', '2023'],
 ['9', '\xa0Singapore', '70,590', '2023'],
 ['10', '\xa0Qatar', '70,070', '2022'],
 ['–', '\xa0Cayman Islands (UK)', '68,790', '2022'],
 ['11', '\xa0Australia', '63,140', '2023'],
 ['12', '\xa0Sweden', '61,650', '2023'],
 ['13', '\xa0Netherlands', '60,670', '2023'],
 ['–', '\xa0Hong Kong (China)', '55,200', '2023'],
 ['14', '\xa0Austria', '55,070', '2023'],
 ['15', '\xa0Israel', '55,020'

In [37]:
f.text.strip().split('\n')

['–', '\xa0Bermuda (UK)', '134,640', '2022']

# 4 - Geolocalizando una IP

**¿Dónde estoy?** --> https://tools.keycdn.com/geo

Esta web nos da toda la información de la IP que introducimos.

In [3]:
import requests as req

In [6]:
url='https://tools.keycdn.com/geo'

In [13]:
html = req.get(url).content

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

WTF? Que ha pasado



que es un [user agent] ??(https://www.zenrows.com/blog/user-agent-web-scraping#what-is)

User agents for web scraping --> [link](https://useragentstring.com/pages/Browserlist/)

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

response = req.get('http://httpbin.org/headers', headers=headers)

print(response.status_code)
print(response.text)

200
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-66e86178-0136ee0767bc1dfe0d426e4b"
  }
}



Se ha tenido que añadir un diccionario que contiene 'User-agent'. Hay muchos sitios web que van a poner impedimentos para que podamos obtener la información a través del scrapeo. Estamos haciendo una petición sin cabecera y no puede identificar el navegador que se está utilizando para acceder.

Cuando accedemos a una web pedimos acceso aportando mucha información en los metadatos: quién soy, dónde estoy...

Al hacer un `get` a pelo no está la información del navegador, se nos detecta como bot y se nos corta el acceso.

Esta información se contiene en lo que se llama **cabecera**. Tal como indico en el diccionario le estoy diciendo a esta web que estoy accediendo desde el navegador Mozilla en su versión 5.0. Estoy intentando sacar información de la web que, a priori, no debería poder sacar. Esta web pone un impedimento para ello, pero hay una solución, la cabecera (el **`header=`**).


In [11]:
from bs4 import BeautifulSoup as bs

In [14]:
http://httpbin.org/headers

SyntaxError: invalid syntax (832054228.py, line 1)

In [15]:
https://useragentstring.com/pages/Browserlist/

SyntaxError: invalid syntax (1069699194.py, line 1)

In [16]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

response = requests.get('http://httpbin.org/headers', headers=headers)

print(response.status_code)
print(response.text)


200
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-66e8623a-1040d814611a23a72c5d5741"
  }
}



In [17]:
# soup parseada para tener el HTML legible
soup = bs(html, 'html.parser')

NameError: name 'html' is not defined

In [ ]:
print(soup.prettify()[:1000]) # con .prettify() lo que hago es poner el código todavía más legible, con indentación
                       # .prettify() funciona distinto dentro que fuera de la función print()

Utilizando el inspector en el navegador podemos activar el cursor y encontrar en qué parte del código HTML se encuentra el elemento que necesitamos. Una vez identificado lo buscamos en nuestra *sopa*

In [ ]:
print(soup.find('div', {'id': 'geoResult'}).prettify()) # conseguido con el inspector

In [ ]:
tabla = soup.find('div', {'id': 'geoResult'}) # le llamo tabla, pues acabará siendo nuestra tabla
info = tabla.find('div', {'class': 'bg-light medium rounded p-3'}) # la información que tendrá la tabla está aquí

Ahora que ya sé dónde está la información que necesito y tengo una variable que la alberga puedo buscar más profundamente hasta poder extraer los datos que necesito.

Toda la info la tengo dentro de ese div veo que lo tengo en una lista con la class='row mb-0'

Pero cuidado porque hay dos clases iguales.

Accedamos a info para que nos dé todos los dd que tenga la clase col-8 text-monospace. Así me va a dar todos los que hay con la misma etiqueta y la misma clase.

Por comodidad de lectura del código esto lo guardamos en una variable que se llama detalles, al fin y al cabo son los detalles de la información que contendrá la tabla que estamos haciendo.

In [ ]:
detalles = info.find_all('dd', {'class': 'col-8 text-monospace'}) #Elemento y añado tb la clase

Hago una lista con el texto que hay en cada elemento de detalles. Le llamo conexión pues son los detalles de la info de esa conexión IP.

In [ ]:
conexion = [e.text for e in detalles]
conexion

Necesito hacer un diccionario. `<dd>` es la información de la conexión. `<dt>` tiene el nombre de cada una de las filas (es la columna de la izquierda). Por ello cojo esa información y lo uso como `keys`, mientras que lo que hemos hecho hasta ahora serán los `values`.

In [ ]:
keys = [e.text for e in tabla.find_all('dt')]
keys

In [ ]:
dicc = {k:v for k,v in zip(keys,conexion)} #me hago un diccionario con los datos de la conexion

In [ ]:
dicc

In [ ]:
import pandas as pd
ip = pd.DataFrame(dicc, index = [0]) #al ser valores escalares hay que proporcionar índice
                                     #al aportar índice se genera una única fila
ip.T

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [ ]:
#Mi url ha cambiado
https://tools.keycdn.com/geo?host=88.30.60.7

In [18]:
url='https://tools.keycdn.com/geo?host=137.255.90.7' #tengo parámetros en la direccion en funcion de la IP que busque...🤔

html = req.get(url, headers = user_agent).content

soup=bs(html, 'html.parser')

NameError: name 'user_agent' is not defined

Es igual que antes, pues la estructura es la misma, primero saco todos los `<dd>`luego todos los `<dt>`

In [ ]:
tabla = soup.find('div', {'id': 'geoResult'})
tabla.find_all('dd', {'class': 'col-8 text-monospace'})

In [ ]:
tabla.find_all('dt')

In [ ]:
#'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

Puedo hacer una función que haga de golpe todo lo que hemos hecho hasta ahora con una dirección IP

In [ ]:
def geo(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = user_agent).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    conexion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

Vimos que una url también puede tener diferentes parámetros. ¿Qué pasa si tengo una lista de IP de la que quiero obtener la información y hacerme un dataframe?

In [ ]:
list_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

In [ ]:
for ip in list_ip:
    print(geo(ip))


In [ ]:
s# Inténtalo tú (alumnos)

### Ejemplo LinkedIn

In [ ]:
#Vamos intentar scrapear Linkedin
URL='https://www.linkedin.com/jobs/search/' #Esta es la direccion de la pagina de busqueda de empleo de Linkedin general

A esta direccion le podemos poner parametros para afinar nuestra búsqueda.

URL = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Madrid&refresh=true'

`?keywords=data` nos dice que la palabra clave que queremos buscar es 'data'
    
`&location=Madrid` nos dice que queremos buscar en Madrid
    
`&refresh=true` nos dice que queremos que nos devuelva los resultados más recientes

Podemos filtrar más nuestra búsqueda en la web y veremos que van apareciendo más ***parametros*** en la URL.

`&f_TPR=r120960` nos dice que empleos de la ultima semana. El parámetro va en segundos así que 60 * 60 * 24 * número de días atrás que queremos buscar

`&start={i*25}` donde i sería el número de página


In [ ]:
#Vamos a intentar scrapear esta URL
import requests as req
from bs4 import BeautifulSoup as bs
url = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Espa%C3%B1a&refresh=true%27&position=1&pageNum=0'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
# Intentad sacar los ingredientes de la recete de esta sopa y obtener para cada oferta de trabajo:

# - Titulo
# - Empresa
# - Ubicacion
# - Link de la empresa
# - Link de la oferta
# - Fecha de publicacion

In [ ]:
#Os recomiendo ir echando una ojeada al html y la sopa poco a poco para ver como esta estructurado
#Lo primero que tendremos que buscar será el elemento que contiene todas las ofertas....🤔

#PISTA -- base-search-card__info
#De aqui en adelante vosotros solos, tened presente que querremos guardar toda esa info en un dataframe

In [ ]:
h3_titulos = soup.find_all('h3',{'class': 'base-search-card__title'})
titulos_ofertas = [h3.text.replace('\n','').strip() for h3 in h3_titulos]
titulos_ofertas

In [ ]:
titulos_ofertas[0].replace('\n','').strip()

In [ ]:
# - Titulo
# - Empresa
# - Ubicacion
# - Link de la empresa
# - Link de la oferta
# - Fecha de publicacion

In [ ]:
# SOLUCION para el primer elemento que aparece
import pandas as pd

data=[]
tabla = soup.find('div', {'class': 'base-search-card__info'})

titulo = tabla.find('h3', {'class': 'base-search-card__title'}).text.split('–')[0].strip() # - Titulo .sprit() para quitar espacios
empresa = tabla.find('h4', {'class': 'base-search-card__subtitle'}).text.strip()  # - Empresa
ubicacion = tabla.find('span', {'class': 'job-search-card__location'}).text.strip() # - Ubicacion
link_empresa = tabla.find('a', {'class': 'hidden-nested-link'})['href'].split('?')[0] # - Link de la empresa
link_oferta = soup.find('a', {'class': 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'})['href'] # - Link de la oferta
fecha_publicacion = tabla.find('time')['datetime'].strip()  # - Fecha de publicacion

print(titulo)
print(empresa)
print(ubicacion)
print(link_empresa)
print(link_oferta)
print(fecha_publicacion)

data.append({
    'titulo': titulo,
    'empresa': empresa,
    'ubicacion': ubicacion,
    'link_empresa': link_empresa,
    'link_oferta': link_oferta,
    'fecha': fecha_publicacion
})

display(pd.DataFrame(data))

### REDDIT - https://www.reddit.com/r/Python/

In [ ]:
url = 'https://www.reddit.com/r/Python/'
html = req.get(url).content
soup = bs(html, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
articulos = soup.find_all('shreddit-post')
print(articulos[0].prettify())

In [ ]:
len(articulos)

Ya sé que tengo el autor del post dentro de `<span class="whitespace-nowrap">` y quiero conseguir el nombre de los autores de los diferentes posts de reddit

soup.find_all('span', {'class':'whitespace-nowrap'}) me va a dar una lista

pues por cada elemento de la lista dame el texto y límpialo. (hemos visto que cada autor empieza por 'u/' y eso nos da igual).

In [ ]:
#Lista Autores
autores =[autor.text.replace('u/','') for autor in soup.find_all('span', {'class':'whitespace-nowrap'})]
autores[2]

In [ ]:
#Titulos:
titulos = []
listado = soup.find_all('a', {'font-bold'})

#este find_all me devuelve artículo y autor [0] no es lo que quiero, pero [1] sí es lo que quiero
#necesito quedarme los impares
for i in range(len(listado)):
  if i%2 != 0:
    titulos.append(listado[i].text.replace('\n','').strip())
titulos

In [ ]:
#Tipo articulo
tipos = [tipo.text.replace('\n','').strip() for tipo in soup.find_all('div',{'class': 'md'})]

Cuando tengo una etiqueta sea la que sea dentro tengo su clase, id, y más atributos. href, title, target...

El contenido de los atributos también es accesible. Es como coger información de lo que pone por fuera de la caja. Puedo localizar cualquier etiqueta :

In [ ]:
#Queremos coger la info de los atributos
soup.find_all('div', {'class': 'grow overflow-hidden'})[0]#.find('a').attrs#['href']

In [ ]:
soup.find_all('div', {'class': 'grow overflow-hidden'})[0].find('a')

In [ ]:
soup.find_all('div', {'class':'grow overflow-hidden'})[0].find('a').attrs

In [ ]:
soup.find_all('div', {'class':'grow overflow-hidden'})[0].find('a').attrs['href']

### REPASO FUNCIONES

In [ ]:
# repaso funciones

def suma(*args):
    return sum(args)

suma(2, 2, 3, 45, 67, 890)

In [ ]:
def saludar(nombre, lang='es', colega=True):
    s=''

    if colega:
        s='colega!!!'

    if lang=='es':
        print('Hola {} {}'.format(nombre, s))

    else:
        print('Hello {} buddy!!!'.format(nombre))

In [ ]:
def saludar_multiple(*lst, lang='es', colega=True):
    for e in lst:
        saludar(e, lang, colega)

In [ ]:
saludar('Pepe')

In [ ]:
saludar(['Pepe', 'en'])

In [ ]:
saludar(*['Pepe', 'en'])

In [ ]:
saludar_multiple('Ana', 'Pepe', 'Juan', 'Maria', lang='en')

In [ ]:
nombres=['Ana', 'Pepe', 'Juan', 'Maria']

config={'lang': 'es', 'colega': True}

In [ ]:
saludar_multiple(*nombres, **config)

In [ ]:
def function (*args, **kwargs):
    return